In [7]:
import torch
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
from torch import nn

In [59]:
# Put on GPU
device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

# select a preset of images:

imgs_1 = noisy_imgs_1[:1]/255
imgs_2 = noisy_imgs_2[:10000]/255
print(imgs_1.shape)

torch.Size([1, 3, 32, 32])


In [ ]:
class ConvolutionTransposed(object):
    def __init__(self, channels_input, channels_output, kernel_size, stride):
        self.weight = torch.Tensor(channels_output, channels_input, kernel_size, kernel_size)
        
        self.channel_input = channels_input
        #print('input channels', self.channel_input)
        self.kernel_size = kernel_size
        self.stride = stride
        
        #print('weight',self.weight.shape)
        
        
    def forward(self, imgs):
        #print('forward')
        _,_,H,W = imgs.shape
        H_out = (H - 1)*self.stride + self.kernel_size 
        W_out = (W - 1)*self.stride + self.kernel_size 
        
        #print('Hout Wout', H_out, W_out)
        
        self.x = imgs.permute(1, 2, 3, 0).reshape(self.channel_input, -1)
        #print('x',self.x.shape)
        self.y = (self.weight.reshape(self.channel_input, -1)).t().matmul(self.x)
        #print('y',self.y.shape)
        self.y = self.y.reshape(self.y.shape[0], -1, imgs.shape[0])
        #print('y2',self.y.shape)
        self.y = self.y.permute(2, 0, 1)
        
        #print(self.y.shape)
        self.y = fold( self.y, (H_out, W_out), kernel_size=(self.kernel_size,self.kernel_size), stride=self.stride)
        
        return self.y

In [129]:
class Convolution(object):
    def __init__(self, channels_input, channels_output, kernel_size, stride):
        
        self.weight = torch.Tensor(channels_output, channels_input, kernel_size, kernel_size)
        self.kernel_size = kernel_size
        self.stride = stride
        
        #print(self.weight.shape)
        
        
    def forward(self, imgs):
        
        _,_,H,W = imgs.shape
        #print('h w ',H,W)
        self.Hout = (H - self.kernel_size)/self.stride + 1
        self.Wout = (H - self.kernel_size)/self.stride + 1
        #print('Hout Wout', self.Hout, self.Wout)
        
        self.x = unfold(imgs, kernel_size = (self.kernel_size, self.kernel_size), stride = self.stride)
        #print('x',self.x.shape)
        self.y = self.x.transpose(1, 2).matmul(self.weight.view(self.weight.size(0), -1).t()).transpose(1, 2)
        #print('y',self.y.shape)
        self.y = fold(self.y, (int(self.Hout), int(self.Wout)),(1,1), stride = 1)
        
        return self.y
    

    def backward(self):
        return None

    def param(self):
        return None


In [130]:
class Net(object):
    def __init__(self):
        self.conv1 = Convolution(3, 10, kernel_size = 4, stride = 2) # tensor(1,10,15,15)
        self.conv2 = Convolution(10, 10, kernel_size = 5, stride = 2) # tensor (1,10,6,6)
        self.convT1 = ConvolutionTransposed(10, 10, kernel_size = 5, stride = 2)
        self.convT2 = ConvolutionTransposed(10, 32, kernel_size = 4, stride = 2)
        
    def forward(self,x):
        print(x.shape)
        y = self.conv1.forward(x)
        print(y.shape)
        y = self.conv2.forward(y)
        print(y.shape)
        y = self.convT1.forward(y)
        print(y.shape)
        y = self.convT2.forward(y)
        print(y.shape)
        
        return y

    def backward (self, *gradwrtoutput ) :
        return None

    

In [133]:
model = Net()


x = torch.ones(1,3,32,32)


y = model.forward(x)


torch.Size([1, 3, 32, 32])
torch.Size([1, 10, 15, 15])
torch.Size([1, 10, 6, 6])
torch.Size([1, 10, 15, 15])
torch.Size([1, 32, 32, 32])


In [124]:
x = torch.ones(1,10,6,6)
#tensor (1,10,6,6)

convT1 = ConvolutionTransposed(10, 10, kernel_size = 5, stride = 2)
convT2 = ConvolutionTransposed(10, 3, kernel_size = 4, stride = 2)

y = convT1.forward(x)
print(y.shape)
y = convT2.forward(y)
print(y.shape)

input channels 10
weight torch.Size([10, 10, 5, 5])
input channels 10
weight torch.Size([3, 10, 4, 4])
forward
Hout Wout 15 15
x torch.Size([10, 36])
y torch.Size([250, 36])
y2 torch.Size([250, 36, 1])
torch.Size([1, 250, 36])
torch.Size([1, 10, 15, 15])
forward
Hout Wout 32 32
x torch.Size([10, 225])
y torch.Size([48, 225])
y2 torch.Size([48, 225, 1])
torch.Size([1, 48, 225])
torch.Size([1, 3, 32, 32])
